In [23]:
import pandas as pd
import catboost
from verstack import NaNImputer
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import keras_tuner as kt
import tensorflow_addons as tfa



In [24]:
data = pd.read_csv('flow_runs.csv', index_col=0)
data

,accuracy,algorithm,task,AutoCorrelation,CfsSubsetEval_DecisionStumpAUC,CfsSubsetEval_DecisionStumpErrRate,CfsSubsetEval_DecisionStumpKappa,CfsSubsetEval_NaiveBayesAUC,CfsSubsetEval_NaiveBayesErrRate,CfsSubsetEval_NaiveBayesKappa,...,RandomTreeDepth2AUC,RandomTreeDepth2ErrRate,RandomTreeDepth2Kappa,RandomTreeDepth3AUC,RandomTreeDepth3ErrRate,RandomTreeDepth3Kappa,StdvNominalAttDistinctValues,kNN1NAUC,kNN1NErrRate,kNN1NKappa
0,0.981401,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
1,0.979748,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
2,0.977268,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
3,0.984088,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
4,0.979955,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444216,0.833224,Ensemble,7293,0.720342,0.632301,0.162751,0.092566,0.632301,0.162751,0.092566,...,0.565331,0.247879,0.128400,0.565331,0.247879,0.128400,0.0,0.551255,0.250590,0.102600
444217,0.836338,Ensemble,7293,0.720342,0.632301,0.162751,0.092566,0.632301,0.162751,0.092566,...,0.565331,0.247879,0.128400,0.565331,0.247879,0.128400,0.0,0.551255,0.250590,0.102600
444218,0.834230,Ensemble,7293,0.720342,0.632301,0.162751,0.092566,0.632301,0.162751,0.092566,...,0.565331,0.247879,0.128400,0.565331,0.247879,0.128400,0.0,0.551255,0.250590,0.102600
444219,0.834366,Ensemble,7293,0.720342,0.632301,0.162751,0.092566,0.632301,0.162751,0.092566,...,0.565331,0.247879,0.128400,0.565331,0.247879,0.128400,0.0,0.551255,0.250590,0.102600


In [26]:
imputer = NaNImputer(conservative=True)
imputed_data = imputer.impute(data.groupby(['task', 'algorithm']).mean())
imputed_data

NaNImputer(conservative = True, n_feats = 10,            
           fix_string_nans = True, verbose = True,                
           multiprocessing_load = 3, fill_nans_in_pure_text = True,                    
           drop_empty_cols = True, drop_nan_cols_with_constant = True                        
           feature_selection = correlation)

Dataset dimensions:
 - rows:         7774
 - columns:      108
 - mb in memory: 6.44
 - NaN cols num: 88
--------------------------

Deploy multiprocessing with 12 parallel proceses



/opt/anaconda3/envs/tesi/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/envs/tesi/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/envs/tesi/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/envs/tesi/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use panda

- CfsSubsetEval_DecisionStumpAUC: imputed 59 NaNs
- CfsSubsetEval_NaiveBayesErrRate: imputed 59 NaNs
- CfsSubsetEval_DecisionStumpErrRate: imputed 59 NaNs
- CfsSubsetEval_kNN1NErrRate:    imputed 59 NaNs
- CfsSubsetEval_kNN1NAUC:        imputed 59 NaNs
- CfsSubsetEval_NaiveBayesKappa: imputed 59 NaNs
- CfsSubsetEval_kNN1NKappa:      imputed 59 NaNs
- CfsSubsetEval_DecisionStumpKappa: imputed 59 NaNs
- CfsSubsetEval_NaiveBayesAUC:   imputed 59 NaNs
- ClassEntropy:                  imputed 16 NaNs
- DecisionStumpAUC:              imputed 59 NaNs
- DecisionStumpErrRate:          imputed 59 NaNs
- CfsSubsetEval_DecisionStumpAUC: imputed 59 NaNs
- CfsSubsetEval_DecisionStumpErrRate: imputed 59 NaNs
- CfsSubsetEval_NaiveBayesErrRate: imputed 59 NaNs
- CfsSubsetEval_kNN1NErrRate:    imputed 59 NaNs
- CfsSubsetEval_NaiveBayesKappa: imputed 59 NaNs
- CfsSubsetEval_kNN1NAUC:        imputed 59 NaNs
- CfsSubsetEval_DecisionStumpKappa: imputed 59 NaNs
- CfsSubsetEval_kNN1NKappa:      imputed 59 NaN

accuracy  AutoCorrelation  \
task   algorithm                                       
1      Ensemble            0.958165         0.606466   
       LinearRegression    0.671121         0.606466   
       LogisticRegression  0.988864         0.606466   
       NaiveBayes          0.974715         0.606466   
       NeuralNetwork       0.980407         0.606466   
...                             ...              ...   
167140 Ensemble            0.912264         0.375510   
       LogisticRegression  0.938046         0.375510   
       SVM                 0.808390         0.375510   
167141 Ensemble            0.925192         0.758352   
       SVM                 0.890160         0.758352   

                           CfsSubsetEval_DecisionStumpAUC  \
task   algorithm                                            
1      Ensemble                                  0.981204   
       LinearRegression                          0.981204   
       LogisticRegression                        0.981204   
       NaiveBayes                                0.981204   
       NeuralNetwork                             0.981204   
...                                                   ...   
167140 Ensemble                                  1.000035   
       LogisticRegression                        1.000035   
       SVM                                       1.000035   
167141 Ensemble                                  1.000035   
       SVM                                       1.000035   

                           CfsSubsetEval_DecisionStumpErrRate  \
task   algorithm                                                
1      Ensemble                                      0.023385   
       LinearRegression                              0.023385   
       LogisticRegression                            0.023385   
       NaiveBayes                                    0.023385   
       NeuralNetwork                                 0.023385   
...                                                       ...   
167140 Ensemble                                      0.758119   
       LogisticRegression                            0.758119   
       SVM                                           0.758119   
167141 Ensemble                                      0.758119   
       SVM                                           0.758119   

                           CfsSubsetEval_DecisionStumpKappa  \
task   algorithm                                              
1      Ensemble                                    0.941800   
       LinearRegression                            0.941800   
       LogisticRegression                          0.941800   
       NaiveBayes                                  0.941800   
       NeuralNetwork                               0.941800   
...                                                     ...   
167140 Ensemble                                    0.999974   
       LogisticRegression                          0.999974   
       SVM                                         0.999974   
167141 Ensemble                                    0.999974   
       SVM                                         0.999974   

                           CfsSubsetEval_NaiveBayesAUC  \
task   algorithm                                         
1      Ensemble                               0.981204   
       LinearRegression                       0.981204   
       LogisticRegression                     0.981204   
       NaiveBayes                             0.981204   
       NeuralNetwork                          0.981204   
...                                                ...   
167140 Ensemble                               1.000048   
       LogisticRegression                     1.000048   
       SVM                                    1.000048   
167141 Ensemble                               1.000048   
       SVM                                    1.000048   

                           CfsSubsetEval_NaiveBayesErrRate  \
task   algorithm     

In [27]:
def test_train_split(dataset, train_size_tasks=0.8, one_hot=False):
    X = dataset.drop(['accuracy'], axis=1)
    y = dataset['accuracy']
    task_list = dataset.index.levels[0].to_list()
    train_tasks = random.sample(task_list, int(train_size_tasks*len(task_list)))
    test_tasks = list(set(task_list)-set(train_tasks))
    X_train = X.loc[train_tasks].reset_index().drop(['task'], axis=1)
    y_train = y.loc[train_tasks].reset_index().drop(['task', 'algorithm'], axis=1)
    X_test = X.loc[test_tasks].reset_index().drop(['task'], axis=1)
    y_test = y.loc[test_tasks].reset_index().drop(['task', 'algorithm'], axis=1)
    if one_hot:
        X_train = pd.concat([X_train.drop(['algorithm'], axis=1), pd.get_dummies(X_train['algorithm'])], axis=1)
        X_test = pd.concat([X_test.drop(['algorithm'], axis=1), pd.get_dummies(X_test['algorithm'])], axis=1)
    return X_train, y_train, X_test, y_test

In [28]:
from sklearn.preprocessing import StandardScaler


X_train, y_train, X_test, y_test = test_train_split(imputed_data, one_hot=True)
X_train[X_train.columns] = StandardScaler().fit_transform(
    X_train[X_train.columns])
X_test[X_test.columns] = StandardScaler().fit_transform(X_test[X_test.columns])

In [ ]:

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))
    for n_layer in range(hp.Int('n_layers', 1, 6, default=2)):
        model.add(
            tf.keras.layers.Dense(hp.Int('n_units_' + str(n_layer), 1, 512),
                               activation=hp.Choice(
                                   'activation', ['relu', 'sigmoid', 'tanh'])))
        model.add(
            tf.keras.layers.Dropout(
                hp.Float('dropout_' + str(n_layer), 0, 0.5, step=0.05)))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss='mse',
                  optimizer='adam',
                  metrics=[tfa.metrics.RSquare(dtype=tf.float32, y_shape=(1,))])
    return model


tuner = kt.Hyperband(build_model,
                     objective=kt.Objective("val_r_square", "max"),
                     max_epochs=100,
                     directory='/tmp/keras_tuner',
                     project_name='lastduel')
tuner.search(X_train,
             y_train,
             epochs=10,
             validation_split=0.2,
             callbacks=[tf.keras.callbacks.TensorBoard("/tmp/tb_logs")])


Trial 254 Complete [00h 01m 02s]
val_r_square: 0.7623276710510254

Best val_r_square So Far: 0.8233042359352112
Total elapsed time: 00h 32m 38s


In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, best_model.predict(X_test))

0.7888580032836355